In [1]:
import csv
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

import my_functions
%load_ext autoreload
%autoreload 2

## Carga de datos

En esta etapa del proyecto se tomaran los 3 dataset ya trabajados en el ETL, utilizando la información necesaria para conformar otros dataset que contengan únicamente los datos necesarios para procesar cada una de las consultas solicitadas para al API.

Se realiza la carga de los dataset df_games, df_reviews y df_items en dataframes.

In [2]:
# Cargar el archivo CSV en un DataFrame
df_games = pd.read_csv('Datasets/Datasets ETL/steam_games_v2.csv')

In [3]:
# Cargar el archivo CSV en un DataFrame
df_reviews = pd.read_csv('Datasets/Datasets ETL/user_reviews_v1.csv')

In [4]:
# Cargar el archivo CSV en un DataFrame
df_items = pd.read_csv('Datasets/Datasets ETL/user_items_v2.csv')

Se examinan brevemente para verificar la carga.

In [5]:
# Mostrar primera fila del DataFrame
df_games.head(1)

,app_name,game_id,developer,release_year,gnr_Action,gnr_Adventure,gnr_Casual,gnr_Early Access,gnr_Free to Play,gnr_Indie,gnr_Massively Multiplayer,gnr_RPG,gnr_Racing,gnr_Simulation,gnr_Sports,gnr_Strategy
0,Lost Summoner Kitty,761140,Kotoshiro,2018,1,0,1,0,0,1,0,0,0,1,0,1


In [6]:
my_functions.describe_df(df_games)

Cantidad Registros:  31332
Cantidad Campos:  16


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,app_name,object,0,0.0,31332,100.0,31296,99.89
1,game_id,int64,0,0.0,31332,100.0,31332,100.00
2,developer,object,0,0.0,31332,100.0,10754,34.32
3,release_year,int64,0,0.0,31332,100.0,44,0.14
4,gnr_Action,int64,0,0.0,31332,100.0,2,0.01
5,gnr_Adventure,int64,0,0.0,31332,100.0,2,0.01
6,gnr_Casual,int64,0,0.0,31332,100.0,2,0.01
7,gnr_Early Access,int64,0,0.0,31332,100.0,2,0.01
8,gnr_Free to Play,int64,0,0.0,31332,100.0,2,0.01
9,gnr_Indie,int64,0,0.0,31332,100.0,2,0.01


In [7]:
# Modificar tipo de columna 'id' a string
df_games['game_id'] = df_games['game_id'].astype(str)

In [8]:
# Mostrar la primera fila del DataFrame
df_reviews.head(1)

,game_id,recommend,user_id,posted_year,sentiment
0,1250,True,76561197970982479,2011,2


In [9]:
my_functions.describe_df(df_reviews)

Cantidad Registros:  58431
Cantidad Campos:  5


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,game_id,int64,0,0.0,58431,100.0,3682,6.30
1,recommend,bool,0,0.0,58431,100.0,2,0.00
2,user_id,object,0,0.0,58431,100.0,25458,43.57
3,posted_year,int64,0,0.0,58431,100.0,7,0.01
4,sentiment,int64,0,0.0,58431,100.0,3,0.01


In [10]:
# Mostra la primera fila del DataFrame
df_items.head(1)

,game_id,app_name,playtime_forever,user_id
0,10,Counter-Strike,6,76561197970982479


In [11]:
my_functions.describe_df(df_items)

Cantidad Registros:  4568172
Cantidad Campos:  4


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,game_id,int64,0,0.0,4568172,100.0,10854,0.24
1,app_name,object,0,0.0,4568172,100.0,10823,0.24
2,playtime_forever,int64,0,0.0,4568172,100.0,9568,0.21
3,user_id,object,0,0.0,4568172,100.0,70827,1.55


## Creacion de DataFrames auxiliares

En este punto se comenzará con el proceso para obtener los dataset finales, para lo cual se crearán algunos dataframe auxiliares.

## df_play_gnr_top

Este dataframe contendrá los campos 'genre', 'release_year'	y 'playtime' agrupados. Es decir, contiene los valores totales de tiempo de juego ('playtime'), para cada género en un año dado.

El mismo será utilizado en la función que se creará más adelante 'def PlayTimeGenre', que responderá a consultas sobre el año con más horas jugadas para cada género.

Para ello se utilizan campos de df_games y df_items, generando el datafram con la información agrupada como se requiere para el objetivo.

In [12]:
# Modificar tipo de columna 'id' a string
df_games['game_id'] = df_games['game_id'].astype(int)

# Realizar la fusión (merge) entre df_games y df_items en base al campo game_id
merged_df = pd.merge(df_games, df_items, on='game_id', how='inner')

# Inicializar un diccionario para almacenar los datos
playtime_data = {'genre': [], 'release_year': [], 'playtime': []}

# Obtener la lista de géneros (columnas que comienzan con prefijo 'gnr_')
genres = [col for col in df_games.columns if col.startswith('gnr_')]

# Iterar sobre los géneros
for genre in genres:
    # Filtrar df_games para obtener los game_id con el género actual
    genre_game_ids = df_games[df_games[genre] == 1]['game_id']

    # Filtrar el DataFrame fusionado por los game_id con el género actual
    genre_data = merged_df[merged_df['game_id'].isin(genre_game_ids)]

    # Agrupar por género y año, sumar el playtime_forever
    grouped_data = genre_data.groupby(['release_year']).agg({'playtime_forever': 'sum'}).reset_index()

    # Agregar datos al diccionario
    for index, row in grouped_data.iterrows():
        # Quitar el prefijo 'gnr_' de la columna genre
        genre_name = genre.replace('gnr_', '')
        playtime_data['genre'].append(genre_name)
        playtime_data['release_year'].append(row['release_year'])
        playtime_data['playtime'].append(row['playtime_forever'])

# Crear el DataFrame df_play_gnr
df_play_gnr = pd.DataFrame(playtime_data)

# Ordenar por género alfabéticamente y playtime de manera descendente
df_play_gnr = df_play_gnr.sort_values(by=['genre', 'playtime'], ascending=[True, False])

# Restablecer el índice después de la clasificación
df_play_gnr = df_play_gnr.reset_index(drop=True)

Se verifica la carga de los datos visualizando los primeros 3 resultados para cada género, que ya fueron ordenados por el campo 'playtime' de mayor a menor.

In [13]:
pd.set_option('display.max_rows', None)

# Imprimir los primeros 3 resultados para cada género
df_play_gnr.groupby('genre').head(3)

,genre,release_year,playtime
0,Action,2012,135335252
1,Action,2011,94291674
2,Action,2013,92350609
34,Adventure,2011,50293919
35,Adventure,2013,49130859
36,Adventure,2015,41066104
69,Casual,2017,22975309
70,Casual,2015,20129459
71,Casual,2014,12895672
99,Early Access,2013,18885459


Por último, se genera un nuevo dataframe con la información exclusivamente necesaria para responder a la consulta y se guarda en el archivo 'df_play_gnr_top.csv'.

In [14]:
# Crear nuevo dataframe primer resultado para cada género
df_play_gnr_top = df_play_gnr.groupby('genre').head(1).reset_index(drop=True)

# Mostrar dataframe
pd.reset_option('display.max_rows')
df_play_gnr_top

,genre,release_year,playtime
0,Action,2012,135335252
1,Adventure,2011,50293919
2,Casual,2017,22975309
3,Early Access,2013,18885459
4,Free to Play,2017,37809857
5,Indie,2013,52790602
6,Massively Multiplayer,2013,22822044
7,RPG,2011,45961909
8,Racing,2015,8362051
9,Simulation,2006,40260140


In [15]:
# Exportar a formato XLSX
# df_play_gnr_top.to_excel('Datasets/Datasets final/df_play_gnr.xlsx', index=False)

# Exportar a formato CSV
df_play_gnr_top.to_csv('Datasets/Datasets final/df_play_gnr.csv', index=False)

# Exportar a formato Parquet
# df_play_gnr_top.to_parquet('Datasets/Datasets final/df_play_gnr.parquet', index=False)

## df_usr_gnr_top / df_usr_gnr_top_year

Se crearán 2 dataframes referidos a tiempo de juego de los usuarios por género.
El primero ('df_usr_gnr_top'), con datos de el usuario con más tiempo de juego para cada género.
El segundo ('df_usr_gnr_top_year'), para cada usuario referido en el dataframe anterior, contendrá los tiempos de juego por año para dicho género.

Primero se conforma un dataframe auxiliar con información de 'df_games' y 'df_items'.

In [16]:
# Hacer un merge entre df_items y df_games y seleccionar campos específicos
df_usr_gnr = pd.merge(df_items, df_games[['game_id', 'release_year'] + [col for col in df_games.columns if col.startswith('gnr_')]], on='game_id', how='inner')[['user_id', 'game_id', 'playtime_forever', 'release_year'] + [col for col in df_games.columns if col.startswith('gnr_')]]

# Mostrar df_usr_items_gnr
df_usr_gnr.head()

,user_id,game_id,playtime_forever,release_year,gnr_Action,gnr_Adventure,gnr_Casual,gnr_Early Access,gnr_Free to Play,gnr_Indie,gnr_Massively Multiplayer,gnr_RPG,gnr_Racing,gnr_Simulation,gnr_Sports,gnr_Strategy
0,76561197970982479,10,6,2000,1,0,0,0,0,0,0,0,0,0,0,0
1,js41637,10,0,2000,1,0,0,0,0,0,0,0,0,0,0,0
2,Riot-Punch,10,0,2000,1,0,0,0,0,0,0,0,0,0,0,0
3,doctr,10,93,2000,1,0,0,0,0,0,0,0,0,0,0,0
4,corrupted_soul,10,108,2000,1,0,0,0,0,0,0,0,0,0,0,0


Se procede a generar registros diferenciados por género para cada juego. Si un juego contiene más de un género, se multiplicarán los registros.

In [17]:
# Utilizar melt para desplegar las columnas de género
df_usr_gnr = pd.melt(df_usr_gnr, id_vars=['user_id', 'game_id', 'release_year', 'playtime_forever'], var_name='genre', value_name='genre_value')

# Filtrar las filas con valor 1 en la columna de género
df_usr_gnr = df_usr_gnr[df_usr_gnr['genre_value'] == 1]

# Filtrar las filas con valor 0 en la columna playtime
df_usr_gnr = df_usr_gnr[df_usr_gnr['playtime_forever'] != 0]

# Eliminar la columna de valor de género
df_usr_gnr = df_usr_gnr.drop(columns=['genre_value'])

# Quitar el prefijo 'gnr_' de la columna genre
df_usr_gnr['genre'] = df_usr_gnr['genre'].str.replace('gnr_', '')

# Agrupar por user_id, release_year y genre, sumando playtime_forever
df_usr_gnr = df_usr_gnr.groupby(['user_id', 'release_year', 'genre']).agg({'playtime_forever': 'sum'}).reset_index()

# Ordenar el DataFrame
df_usr_gnr = df_usr_gnr.sort_values(by=['genre', 'playtime_forever'], ascending=[True, False]).reset_index(drop=True)

# Mostrar df_usr_gnr
df_usr_gnr.head(5)

,user_id,release_year,genre,playtime_forever
0,76561198281817428,2015,Action,19788
1,76561198100195552,2012,Action,19380
2,awdawdawdawdawda,2017,Action,19139
3,ninjaoficial2,2017,Action,19018
4,76561198061882668,2012,Action,18827


Se verifica la estrucutra del dataframe.

In [18]:
df_usr_gnr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2656430 entries, 0 to 2656429
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   release_year      int64 
 2   genre             object
 3   playtime_forever  int64 
dtypes: int64(2), object(2)
memory usage: 81.1+ MB


In [19]:
df_usr_gnr['genre'].unique()

array(['Action', 'Adventure', 'Casual', 'Early Access', 'Free to Play',
       'Indie', 'Massively Multiplayer', 'RPG', 'Racing', 'Simulation',
       'Sports', 'Strategy'], dtype=object)

In [20]:
# Exportar a formato XLSX
# df_usr_gnr.to_excel('Datasets/df_usr_gnr.xlsx', index=False)

# Exportar a formato CSV
# df_usr_gnr.to_csv('Datasets/Datasets merge/df_usr_gnr.csv', index=False)

# Exportar a formato Parquet
# df_usr_gnr.to_parquet('Datasets/Datasets merge/df_usr_gnr.parquet', index=False)

A continuación se agrupan los datos por usuario y género para obtener posteriormente el usuario con más horas jugada para cada género.

In [21]:
# Agrupar por user_id, y genre, sumando playtime_forever
df_usr_gnr_total = df_usr_gnr.groupby(['user_id', 'genre']).agg({'playtime_forever': 'sum'}).reset_index().sort_values(by=['genre', 'playtime_forever'], ascending=[True, False])

# Mostrar df_usr_gnr
df_usr_gnr_total

,user_id,genre,playtime_forever
469382,awdawdawdawdawda,Action,26105
94824,76561198045497695,Action,26094
52654,76561198027114566,Action,25960
216985,76561198072277306,Action,25941
308967,76561198093535711,Action,25941
...,...,...,...
594621,vipersun,Strategy,1
594676,virutso,Strategy,1
600817,yeahno,Strategy,1
600859,yel369,Strategy,1


In [22]:
# Exportar a formato XLSX
# df_usr_gnr_total.to_excel('Datasets/Datasets merge/df_usr_gnr_total.xlsx', index=False)

# Exportar a formato CSV
# df_usr_gnr_total.to_csv('Datasets/Datasets merge/df_usr_gnr_total.csv', index=False)

# Exportar a formato Parquet
# df_usr_gnr_total.to_parquet('Datasets/Datasets merge/df_usr_gnr_total.parquet', index=False)

Utilizando este último dataframe se extraen los datos necesario para responder la consulta y se guardan en 'df_usr_gnr_top.csv'.

In [23]:
# Copiar df_usr_gnr_total a df_usr_gnr_top
df_usr_gnr_top = df_usr_gnr_total.groupby('genre').first().reset_index()

# Mostrar df_usr_gnr_top
df_usr_gnr_top


,genre,user_id,playtime_forever
0,Action,awdawdawdawdawda,26105
1,Adventure,76561198068546969,23255
2,Casual,76561198159037032,21592
3,Early Access,Iassassinatedkennedy,15884
4,Free to Play,76561198068226047,24254
5,Indie,76561198093572344,25426
6,Massively Multiplayer,76561198083020760,22171
7,RPG,76561198079373395,25413
8,Racing,SubieTec,16462
9,Simulation,76561198080164247,23059


In [24]:
# Exportar a formato XLSX
# df_usr_gnr_top.to_excel('Datasets/Datasets final/df_usr_gnr_top.xlsx', index=False)

# Exportar a formato CSV
df_usr_gnr_top.to_csv('Datasets/Datasets final/df_usr_gnr_top.csv', index=False)

# Exportar a formato Parquet
# df_usr_gnr_top.to_parquet('Datasets/Datasets final/df_usr_gnr_top.parquet', index=False)

Por último, se genera 'df_usr_gnr_top_year.csv' con la información de tiempo de juego por año de los usuarios con más horas de juego para cada género.

Este dataset contendrá información de tiempo de juego por año ('playtime_forever') y el tiempo de juego acumulado a lo largo de los años ('playtime_acumulated').

In [25]:
# Copiar df_usr_gnr_top a df_usr_gnr_top_year
df_usr_gnr_top_year = pd.merge(df_usr_gnr_top[['user_id', 'genre']], df_usr_gnr[['user_id', 'genre', 'release_year', 'playtime_forever']], on=['user_id', 'genre'], how='left')

# Ordenar el DataFrame
df_usr_gnr_top_year = df_usr_gnr_top_year.sort_values(by=['genre', 'release_year'], ascending=[True, True])

# Crear una nueva columna playtime_acumulated que sume el acumulado por género y usuario
df_usr_gnr_top_year['playtime_acumulated'] = df_usr_gnr_top_year.groupby(['user_id', 'genre'])['playtime_forever'].cumsum()

df_usr_gnr_top_year = df_usr_gnr_top_year.reset_index()

# Seleccionar las columnas requeridas (user_id, genre y playtime_acumulated)
# df_usr_gnr_top_year = df_usr_gnr_top_year[['user_id', 'genre', 'playtime_acumulated']]

# Mostrar df_usr_gnr_top_year
pd.set_option('display.max_rows', None)
df_usr_gnr_top_year


,index,user_id,genre,release_year,playtime_forever,playtime_acumulated
0,2,awdawdawdawdawda,Action,2009,1552,1552
1,3,awdawdawdawdawda,Action,2012,44,1596
2,1,awdawdawdawdawda,Action,2014,5366,6962
3,4,awdawdawdawdawda,Action,2015,4,6966
4,0,awdawdawdawdawda,Action,2017,19139,26105
5,6,76561198068546969,Adventure,2011,7421,7421
6,7,76561198068546969,Adventure,2014,811,8232
7,5,76561198068546969,Adventure,2016,15003,23235
8,8,76561198068546969,Adventure,2017,20,23255
9,18,76561198159037032,Casual,2006,21,21


In [26]:
# Exportar a formato XLSX
# df_usr_gnr_top_year.to_excel('Datasets/Datasets final/df_usr_gnr_top_year.xlsx', index=False)

# Exportar a formato CSV
df_usr_gnr_top_year.to_csv('Datasets/Datasets final/df_usr_gnr_top_year.csv', index=False)

# Exportar a formato Parquet
# df_usr_gnr_top_year.to_parquet('Datasets/Datasets final/df_usr_gnr_top_year.parquet', index=False)

## df_games_rec

Este dataframe auxiliar servirá de base para procesar la información necesaria de la consulta 'def UsersRecommend', que debe devolver el top 3 de juegos más recomendados para el año dado.
Se define que un juego es recomendado por un usuario a partir de los campos 'recommend' (si es 'True') y 'sentiment' (positivos / neutrales) de el dataframe df_reviews.


In [27]:
# Crear df_user_rec con las columnas requeridas
df_games_rec = pd.merge(df_reviews[df_reviews['recommend'] & df_reviews['sentiment'].isin([1, 2])],
                        df_games[['game_id']],
                        on='game_id',
                        how='left')

# Agrupar por game_id y posted_year, calcular total_recommend
df_games_rec = df_games_rec.groupby(['game_id', 'posted_year']).agg(total_recommend=('recommend', 'sum')).reset_index()

# Ordenar el DataFrame
df_games_rec = df_games_rec.sort_values(by=['posted_year', 'total_recommend'], ascending=[True, False])

# Agregar app_name al df_games_rec tomando los app_name de df_games
df_games_rec = pd.merge(df_games_rec, df_games[['game_id', 'app_name']],
                        on='game_id', how='left', suffixes=('', '_games')).reset_index()

# Manejar el caso de game_id en df_reviews que no está en df_games
df_games_rec['app_name'].fillna('Sin datos', inplace=True)

# Mostrar primeros 3 registros por año
df_games_rec.groupby('posted_year').head(3)

,index,game_id,posted_year,total_recommend,app_name
0,0,730,0,584,Counter-Strike: Global Offensive
1,1,440,0,227,Team Fortress 2
2,2,4000,0,128,Garry's Mod
1516,1516,440,2010,9,Team Fortress 2
1517,1517,1250,2010,5,Killing Floor
1518,1518,630,2010,4,Alien Swarm
1553,1553,440,2011,73,Team Fortress 2
1554,1554,620,2011,25,Portal 2
1555,1555,105600,2011,21,Terraria
1729,1729,440,2012,242,Team Fortress 2


In [28]:
# Exportar a formato XLSX
# df_games_rec.to_excel('Datasets/Datasets merge/df_games_rec.xlsx', index=False)

# Exportar a formato CSV
# df_games_rec.to_csv('Datasets/Datasets merge/df_games_rec.csv', index=False)

# Exportar a formato Parquet
# df_games_rec.to_parquet('Datasets/Datasets merge/df_games_rec.parquet', index=False)

## df_games_rec_top_3

Habiendo verificado la correctar carga y ordenamiento de los datos en 'df_usr_rec' se filtran y guardan los datos del top 3 de juegos más recomendados por año en el archivo 'df_usr_rec_top_3.csv'.

In [29]:
pd.set_option('display.max_rows', None)

# Mostrar solo los primeros 3 registros por posted_year y descartar los que tienen total_recommend igual a 0
df_games_rec_top_3 = df_games_rec[df_games_rec['total_recommend'] > 0].groupby('posted_year').head(3).reset_index()

# Mostrar df_games_rec_top_100
df_games_rec_top_3

,level_0,index,game_id,posted_year,total_recommend,app_name
0,0,0,730,0,584,Counter-Strike: Global Offensive
1,1,1,440,0,227,Team Fortress 2
2,2,2,4000,0,128,Garry's Mod
3,1516,1516,440,2010,9,Team Fortress 2
4,1517,1517,1250,2010,5,Killing Floor
5,1518,1518,630,2010,4,Alien Swarm
6,1553,1553,440,2011,73,Team Fortress 2
7,1554,1554,620,2011,25,Portal 2
8,1555,1555,105600,2011,21,Terraria
9,1729,1729,440,2012,242,Team Fortress 2


In [30]:
# Exportar a formato XLSX
# df_games_rec_top_3.to_excel('Datasets/Datasets final/df_games_rec_top_3.xlsx', index=False)

# Exportar a formato CSV
df_games_rec_top_3.to_csv('Datasets/Datasets final/df_games_rec_top_3.csv', index=False)

# Exportar a formato Parquet
# df_games_rec_top_3.to_parquet('Datasets/Datasets final/df_games_rec_top_3.parquet', index=False)

## df_dev_rec_worst_3

En este punto se trabajará sobre la información necesaria para responder a la consulta de la función 'def UsersWorstDeveloper' que debe devolver el top 3 de desarrolladoras con juegos menos recomendados para el año dado, calculado a partir de 'recommend' con valor 'False' y 'sentiment' negativos (valor 0).


In [31]:
# Crear una lista de desarrolladores únicos
developers = df_games['developer'].unique()

# Inicializar un diccionario para almacenar los datos
data_dict = {'developer': [], 'posted_year': [], 'neg_rec': []}

# Iterar sobre los desarrolladores
for dev in developers:
    # Filtrar df_games por el desarrollador actual
    dev_games = df_games[df_games['developer'] == dev]

    # Obtener los años únicos de df_reviews para ese desarrollador
    years = df_reviews[df_reviews['game_id'].isin(dev_games['game_id'])]['posted_year'].unique()

    # Iterar sobre los años
    for year in years:
        # Filtrar df_reviews por el desarrollador y el año actual
        dev_year_reviews = df_reviews[(df_reviews['game_id'].isin(dev_games['game_id'])) & (df_reviews['posted_year'] == year)]

        # Calcular la sumatoria de registros donde recommend es False y sentiment es 0
        neg_rec_count = dev_year_reviews[(dev_year_reviews['recommend'] == False) & (dev_year_reviews['sentiment'] == 0)].shape[0]

        # Agregar datos al diccionario
        data_dict['developer'].append(dev)
        data_dict['posted_year'].append(year)
        data_dict['neg_rec'].append(neg_rec_count)

# Crear el DataFrame df_dev_rec
df_dev_rec = pd.DataFrame(data_dict)

# Eliminar filas con developer igual a 'Sin Datos' o posted_year igual a 0
df_dev_rec = df_dev_rec[(df_dev_rec['developer'] != 'Sin Datos') & (df_dev_rec['posted_year'] != 0)]

# Ordenar el DataFrame por neg_rec de forma descendente
df_dev_rec = df_dev_rec.sort_values(by=['posted_year', 'neg_rec'], ascending=[True, False]).reset_index()

In [32]:
# Imprimir el DataFrame resultante
pd.reset_option('display.max_rows')
df_dev_rec

,index,developer,posted_year,neg_rec
0,18,Valve,2010,0
1,43,Facepunch Studios,2010,0
2,74,Firaxis Games,2010,0
3,95,id Software,2010,0
4,175,Ubisoft Montreal,2010,0
...,...,...,...,...
3036,4261,Synergy Team,2015,0
3037,4266,Empires,2015,0
3038,4272,Black Element,2015,0
3039,4274,Troika Games,2015,0


In [33]:
# Exportar a formato XLSX
# df_dev_rec.to_excel('Datasets/Datasets merge/df_dev_rec.xlsx', index=False)

# Exportar a formato CSV
# df_dev_rec.to_csv('Datasets/Datasets merge/df_dev_rec.csv', index=False)

# Exportar a formato Parquet
# df_dev_rec.to_parquet('Datasets/Datasets merge/df_dev_rec.parquet', index=False)

Se genera el datafram con el top 3 de desarrolladores menos recomendados y se guarda en 'df_dev_rec_worst_3.csv'.

In [34]:
# Guardar solo los primeros 3 registros por posted_year y descartar los que tienen neg_rec igual a 0
df_dev_rec_worst_3 = df_dev_rec[df_dev_rec['neg_rec'] > 0].groupby('posted_year').head(3)

# Mostrar df_dev_rec_worst_3
df_dev_rec_worst_3

,index,developer,posted_year,neg_rec
31,1654,Broken Rules,2011,2
32,17,Valve,2011,1
157,13,Valve,2012,1
158,269,Volition,2012,1
159,638,NeoCoreGames,2012,1
338,16,Valve,2013,10
339,1465,Infinity Ward,2013,5
340,3011,Andrii Vintsevych,2013,5
770,12,Valve,2014,43
771,145,Bohemia Interactive,2014,23


In [35]:
# Exportar a formato XLSX
# df_dev_rec_worst_3.to_excel('Datasets/Datasets final/df_dev_rec_worst_3.xlsx', index=False)

# Exportar a formato CSV
df_dev_rec_worst_3.to_csv('Datasets/Datasets final/df_dev_rec_worst_3.csv', index=False)

# Exportar a formato Parquet
# df_dev_rec_worst_3.to_parquet('Datasets/Datasets final/df_dev_rec_worst_3.parquet', index=False)

In [36]:
def UsersWorstDeveloper(posted_year: int):
    # Filtrar df_dev_rec_worst_3 por el posted_year proporcionado
    year_data = df_dev_rec_worst_3[df_dev_rec_worst_3['posted_year'] == posted_year]

    if not year_data.empty:
        # Presentar mensaje
        print(f"Desarrolladores con más reseñas negativas para el año {posted_year} son:")

        # Mostrar developer y neg_rec para el posted_year dado
        result = year_data[['developer', 'neg_rec']]
        print(result)
    else:
        print(f"No hay datos disponibles para el posted_year {posted_year} en df_dev_rec_worst_3.")

In [37]:
UsersWorstDeveloper(2015)

Desarrolladores con más reseñas negativas para el año 2015 son:
                developer  neg_rec
1741                Valve       42
1742  Bohemia Interactive       28
1743    Facepunch Studios       27


## df_dev_sent

Se trabaja sobre la información necesaria para responder a la consulta de la función 'def sentiment_analysis'. Según la empresa desarrolladora, se debe devolver un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor. 

Para ello se genera el archivo 'df_dev_sent.csv', contabilizando los valores de comentarios analizados con un valor de sentimiento.


In [38]:
# Realizar la fusión (merge) entre df_reviews y df_games en base al campo game_id
merged_df = pd.merge(df_reviews, df_games, on='game_id', how='left')

# Inicializar un diccionario para almacenar los datos
developer_sentiments = {'developer': [], 'negative_sentiment': [], 'neutral_sentiment': [], 'positive_sentiment': [], 'total_sentiment': []}

# Iterar sobre los desarrolladores únicos
for developer in merged_df['developer'].unique():
    # Filtrar el DataFrame fusionado por el desarrollador actual
    dev_reviews = merged_df[merged_df['developer'] == developer]

    # Calcular la sumatoria de registros para cada categoría de sentimiento
    negative_count = dev_reviews[dev_reviews['sentiment'] == 0].shape[0]
    neutral_count = dev_reviews[dev_reviews['sentiment'] == 1].shape[0]
    positive_count = dev_reviews[dev_reviews['sentiment'] == 2].shape[0]
    total_count = len(dev_reviews)

    # Agregar datos al diccionario
    developer_sentiments['developer'].append(developer)
    developer_sentiments['negative_sentiment'].append(negative_count)
    developer_sentiments['neutral_sentiment'].append(neutral_count)
    developer_sentiments['positive_sentiment'].append(positive_count)
    developer_sentiments['total_sentiment'].append(total_count)

# Crear el DataFrame df_dev_sent
df_dev_sent = pd.DataFrame(developer_sentiments)

# Ordenar el DataFrame por developer alfabéticamente descendente
df_dev_sent = df_dev_sent.sort_values(by='developer', ascending=False)

# Imprimir el DataFrame resultante
df_dev_sent


,developer,negative_sentiment,neutral_sentiment,positive_sentiment,total_sentiment
1563,"高考恋爱委员会,橘子班",0,1,1,2
967,橘子班,1,2,1,4
1505,"インレ,Inre",1,1,3,5
465,△○□× (Miwashiba),0,0,5,5
127,xXarabongXx,1,0,0,1
...,...,...,...,...,...
437,11 bit studios,26,9,27,62
522,10tons Ltd,0,0,1,1
1819,"10th Art Studio,Adventure Productions",1,0,1,2
1368,07th Expansion,1,2,0,3


In [39]:
# Exportar a formato XLSX
# df_dev_sent.to_excel('Datasets/Datasets final/df_dev_sent.xlsx', index=False)

# Exportar a formato CSV
df_dev_sent.to_csv('Datasets/Datasets final/df_dev_sent.csv', index=False)

# Exportar a formato Parquet
# df_dev_sent.to_parquet('Datasets/Datasets final/df_dev_sent.parquet', index=False)

### Exportaciones finales

Se guardan todos los dataframes generados para el deploy de la API en la carpeta correspondiente a la misma.

In [40]:
# Exportar a formato CSV
df_play_gnr_top.to_csv('FastAPI/Datasets/df_play_gnr.csv', index=False)
# Exportar a formato CSV
df_usr_gnr_top.to_csv('FastAPI/Datasets/df_usr_gnr_top.csv', index=False)
# Exportar a formato CSV
df_usr_gnr_top_year.to_csv('FastAPI/Datasets/df_usr_gnr_top_year.csv', index=False)
# Exportar a formato CSV
df_games_rec_top_3.to_csv('FastAPI/Datasets/df_games_rec_top_3.csv', index=False)
# Exportar a formato CSV
df_dev_rec_worst_3.to_csv('FastAPI/Datasets/df_dev_rec_worst_3.csv', index=False)
# Exportar a formato CSV
df_dev_sent.to_csv('FastAPI/Datasets/df_dev_sent.csv', index=False)